In [48]:
from janome.tokenizer import Tokenizer
from gensim.models import Word2Vec
from tqdm import tqdm
import torch
from reformer_pytorch import LSHSelfAttention
from reformer_pytorch import LSHAttention

In [8]:
janome_tokenizer = Tokenizer()

In [20]:
def parse():
	sentences = []
	with open("./dialogs/corpus_1.txt", "r") as f:
		contents = f.read().split("\n")
		for content in tqdm(contents[-10000:], leave=False):
			if len(content.split("[SEP]")) > 2:
				_, _, utterance = content.split("[SEP]")
				utterance = [t.surface for t in janome_tokenizer.tokenize(utterance)]
				if len(utterance) > 0:
					sentences.append(utterance)

	return sentences

def train(datas):
	model = Word2Vec(datas, vector_size=128, epochs=10, min_count=0)
	model.save("./models/embedding.w2v")
	return model

def load():
	model = Word2Vec(parse(), vector_size=128, epochs=10, min_count=0)
	model.load("./models/embedding.w2v")
	return model

sentences = parse()
model     = train(sentences)

In [10]:
def get_vector(sentence):
    return [model.wv.get_vector(w) for w in sentence]

def sentence_range(f, e):
    tmp = []
    for i in range(e):
        tmp += sentences[f+i] + ["。"]
        
    return tmp

def get_data(n, r=3):
    i = sentence_range(n, r)
    return i, sentences[n+r+1], get_vector(i), get_vector(sentences[n+r+1])

In [21]:
x, y, x_vec, y_vec = get_data(4346)
print("".join(x))
print("".join(y))

まだ寝てたい …。…zzZ。冷えタオ。
9℃か……暖かいな


In [52]:
attention = LSHAttention(bucket_size=1)

In [55]:
def calculate_attention(n):
    x, y, x_vec, y_vec = get_data(n) # x=v, y=qk yからxを重み付け
    x_vec = torch.tensor([x_vec])
    y_vec = torch.tensor([y_vec])
    
    return attention(x_vec, y_vec)
calculate_attention(4348)

RuntimeError: index 13 is out of bounds for dimension 1 with size 10

検索真ん中はめちゃいいですね。デザイン悪いほうがいいというのは絶対ないと思うのでこういうの増えてほしいなと。なんか洗練された感あります。
重要なお知らせを最も目につく検索の下においてるあたり相当すごい人がデザインしてるのかもしれない（小並感）
上と下をSelfAttention, weight<n以下を0